# 워드 임베딩

## 벡터화

### Bag of words/ DTM(Document-Term Matrix)
- Bag of words : 단어의 순서를 고려하지 않고, 단어의 등장 빈도(frequency)만 고려하여 단어를 벡터화 하는 방법
- DTM : 문서 간 유사도를 비교하기 위한 행렬, 문서=행/단어=열
- 희소 벡터 : 대부분의 값이 0인 벡터
- 단어장 : 중복 카운트를 배제한 단어들의 집합


### TF-IDF
- 단어의 중요도를 계산하기 위한 가중치 기법
- 모든 문서에 자주 등장하는 중요하지 않은 단어(예: the, is 등)의 영향은 줄이고, 특정 문서에만 자주 등장하는 단어의 중요도를 강조
- TF(단어 빈도): 해당 문서에서 단어가 얼마나 자주 등장했는지
- IDF(역문서 빈도): 그 단어가 전체 문서 중 얼마나 희귀한지


### 원-핫 인코딩 구현
- 데이터 전처리 : 텍스트 데이터에서 단어들의 집합인 단어장 생성.
- 단어장의 모든 단어에 대하여 1~V까지 고유한 정수 부여(인덱스 역할)
- 원-핫 벡터 : 원-핫 인코딩을 통해 얻은 벡터
- TF = 문장을 구성하는 단어들의 원-핫 벡터들을 모두 더해서 문장의 개수로 나눈것

#### Step1. 패키지 설치하기

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0;

In [ ]:
import re
from konlpy.tag import Okt
from collections import Counter
print("임포트 완료")

임포트 완료


In [ ]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

#### Step2. 전처리

- 특수 문자(!, ~, .) 제거
- 한국어 정규 표현식  
  - regex : [^ㄱ-ㅎㅏ-ㅣ가-힣 ]

In [ ]:
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


#### Step3. 토큰화

- 단어장 구성을 위함.
- 한국어는 형태소 분석기를 통해 토큰 단위로 나눔

In [ ]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


#### Step4. 단어장 만들기
- 빈도수가 높은 단어일수록 낮은 점수를 부여함.

In [ ]:
vocab = Counter(tokens) #단어의 빈도 체크
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [ ]:
vocab['임금님']

2

In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [ ]:
# vocab에서 각 단어의 첫 번째 요소 word[0] (예: 'apple')를 키로,
# 해당 인덱스에 +1 한 값을 값으로 갖는 딕셔너리를 만드는 것.
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


#### Step5.원-핫 벡터 만들기

In [ ]:
# 특정 단어와 단어장을 입력하면 해당 단어의 원-핫 벡터를 리턴하는 함수
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector
print("슝=3")

슝=3


In [ ]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

- 케라스를 통한 원-핫인코딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
print("임포트 완료")

임포트 완료


In [ ]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [ ]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [ ]:
vocab_size = len(t.word_index) + 1
print("슝=3")

슝=3


- vocab_size +1 이유
 - 자연어 처리 시 0번 단어는 특별 토근으로 단어장 추가가 대부분임.
 - 0번은 padding 작업을 위한 패딩 토큰으로 사용됨.

In [ ]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [ ]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


## 워드 임베딩

📌 원-핫 벡터의 한계

단어를 고유한 정수 인덱스로 표현 → 대부분 0이고 한 개만 1인 희소 벡터(sparse vector).

단어 간 의미적 유사성을 표현할 수 없음.

예: ‘강아지’와 ‘고양이’는 유사하지만, 원-핫 벡터에서는 내적이 0으로 모든 단어가 직교(orthogonal) → 유사도 표현 불가.

따라서 기계는 의미를 이해하거나 새로운 문장을 생성하는 데 어려움이 있음.


📌 워드 임베딩(Word Embedding)의 등장

단어를 고정된 길이의 밀집 벡터(dense vector) 로 표현.

대부분의 값이 0이 아닌 실수값 → 희소 벡터의 반대 개념.

단어 간 의미적 유사성이나 관계를 벡터 공간에서 나타낼 수 있음.

벡터 간 내적, 거리 등을 통해 유사성을 계산할 수 있음.

📌 워드 임베딩의 특징

벡터의 각 요소는 사람 눈에는 의미 해석이 어려우나, 다양한 의미를 내포.

예: 사과 = [0.8, 0.7, 0.7, 0.1] → "둥글고, 빨갛고, 달고, 조금 신" 특성을 내포 (예시일 뿐 실제 벡터는 다름).

기계는 학습을 통해 단어 벡터를 반복적으로 조정하여 의미를 잘 담도록 함.

신경망이나 통계 기반 기법 사용.

마치 딥러닝의 가중치를 학습하는 것처럼.

✅ 결론

워드 임베딩은

1) 단어를 짧은 밀집 벡터로 표현하고,

2) 그 벡터에 의미나 관계를 내포시키는 것이 핵심.

##  Word2Vec
 - 텍스트 데이터 다루기 - Word2Vec 단어로 벡터 만들기 : https://record3329.tistory.com/29

### 분포가설
 -  ‘비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다.’

### CBoW(Continuous Bag of words)
- 정의 : 주변에 있는 단어들을 통해 중간에 있는 단어들을 예측하는 방법



### Skip-Gram
- 중간에 있는 단어로 주변 단어들을 예측하는 방법





### negative sampling
 - 주어진 데이터에서 무작위로 음성(negative) 샘플을 추출하는 방법
 - 양성 샘플과 음성 샘플의 수를 균형있게 맞춰서 모델이 학습할 수 있는 환경을 만듬.
 - 단순히 무작위로 샘플링을 하는 것이 아니라, 더 중요한 샘플에 더 많은 가중치를 부여하여 샘플링 비율을 조절할 수 있음.
 - 모델이 보다 중요한 샘플에 더 집중하도록 함

In [ ]:
import nltk
nltk.download('abc')

## 이부분 punkt 대신 punkt_tab을 이용
nltk.download('punkt_tab')

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.corpus import abc
corpus = abc.sents()
print("슝~")

슝~


In [ ]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [ ]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


- 파라메터의 의미
  - vector size = 학습 후 임베딩 벡터의 차원
  - window = 컨텍스트 윈도우 크기
  - min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않음.)
  - workers = 학습을 위한 프로세스 수
  - sg = 0은 CBoW, 1은 Skip-gram.

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.923514723777771), ('skull', 0.9112710952758789), ('Bang', 0.9060195684432983), ('asteroid', 0.9052225947380066), ('third', 0.9020858407020569), ('baby', 0.8995046615600586), ('dog', 0.8983686566352844), ('bought', 0.8971353769302368), ('rally', 0.8909624218940735), ('dinosaur', 0.8893425464630127)]


In [ ]:
from gensim.models import KeyedVectors

### 코랩이라 경로 바꿔줘야합니다
model.wv.save_word2vec_format('./w2v')
loaded_model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

모델  load 완료!


In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.923514723777771), ('skull', 0.9112710952758789), ('Bang', 0.9060195684432983), ('asteroid', 0.9052225947380066), ('third', 0.9020858407020569), ('baby', 0.8995046615600586), ('dog', 0.8983686566352844), ('bought', 0.8971353769302368), ('rally', 0.8909624218940735), ('dinosaur', 0.8893425464630127)]


- out of vacabuary 문제

In [ ]:
# 에러나는 코드들
# 에러가 나더라도 놀라지 마세요.
loaded_model.most_similar('overacting')
loaded_model.most_similar('memorry')

KeyError: "Key 'overacting' not present in vocabulary"

## 임베딩 벡터의 시각화

In [ ]:
# word2vec 모델 메타정보 및 텐서 내보내기
!python -m gensim.scripts.word2vec2tensor --input ./w2v --output ./w2v

2025-06-17 06:57:21,061 - word2vec2tensor - INFO - running /usr/local/lib/python3.11/dist-packages/gensim/scripts/word2vec2tensor.py --input ./w2v --output ./w2v
2025-06-17 06:57:21,061 - keyedvectors - INFO - loading projection weights from ./w2v
2025-06-17 06:57:21,694 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (10363, 100) matrix of type float32 from ./w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-06-17T06:57:21.686795', 'gensim': '4.3.3', 'python': '3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]', 'platform': 'Linux-6.1.123+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2025-06-17 06:57:22,423 - word2vec2tensor - INFO - 2D tensor file saved to ./w2v_tensor.tsv
2025-06-17 06:57:22,423 - word2vec2tensor - INFO - Tensor metadata file saved to ./w2v_metadata.tsv
2025-06-17 06:57:22,424 - word2vec2tensor - INFO - finished running word2vec2tensor.py


## FastText
- Wore2Vec 이후 등장한 워드 임베딩 방법
- 문자 단위 n-gram(character-level n-gram) 표현을 학습함
- Word2Vec은 더이상 깨질 수 없는 단위로 구분
  
  vs FastText 는 내부 단어(subwords)들을 학습한다

- 오타에 강건(robust)함

- OOV(Out-Of-Vocabulary) 문제 : 기계학습 모델이 훈련 데이터에서는 보지 못했지만, 테스트 데이터에 등장하는 단어를 인식하지 못하는 문제

- **FastText는 OOV 문제를 해결할 수 있음**.
- 왜냐하면 훈련 데이터에서 등장하지 않은 단어라도 서브워드로 분해하면서 해당 서브워드를 포함하고 있는 다른 단어들의 임베딩 벡터를 활용하여 새로운 단어의 임베딩 벡터를 만들기 때문임.

In [ ]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

FastText 학습 완료!


In [ ]:
fasttext_model.wv.most_similar('overacting')

[('extracting', 0.9424561262130737),
 ('shooting', 0.9382485151290894),
 ('lifting', 0.9375076293945312),
 ('fluctuating', 0.9359311461448669),
 ('mixing', 0.9353761672973633),
 ('losing', 0.9350128769874573),
 ('attracting', 0.9310552477836609),
 ('shifting', 0.9307162165641785),
 ('negotiating', 0.9292001724243164),
 ('emptying', 0.9288997054100037)]

In [ ]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9544547200202942),
 ('musical', 0.862307608127594),
 ('intelligence', 0.8490725755691528),
 ('sense', 0.8451464772224426),
 ('mechanical', 0.8398575186729431),
 ('mechanism', 0.8366336226463318),
 ('basic', 0.8358100652694702),
 ('mechanisms', 0.8347879648208618),
 ('sensitivity', 0.831736147403717),
 ('music', 0.8314796090126038)]

## GloVe(글로브, Global Vectors for Word Representation)
 - 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론
 - 특징 : 카운트 기반과 예측 기반 두 가지 방법을 모두 사용

In [ ]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

[==================================================] 100.0% 66.0/66.0MB downloaded


[('cat', 0.9218004941940308),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583713531494),
 ('puppy', 0.7754920721054077),
 ('pet', 0.7724708318710327),
 ('rabbit', 0.7720814347267151),
 ('pig', 0.7490062117576599),
 ('snake', 0.7399188876152039),
 ('baby', 0.7395570278167725),
 ('bite', 0.7387937307357788)]

In [ ]:
glove_model.most_similar('overacting')

[('impudence', 0.7842012047767639),
 ('puerile', 0.7816032767295837),
 ('winningly', 0.7644237875938416),
 ('grossness', 0.7576098442077637),
 ('deconstructions', 0.748936653137207),
 ('over-the-top', 0.7460805773735046),
 ('buffoonery', 0.746045708656311),
 ('impetuosity', 0.7415392398834229),
 ('sophomoric', 0.736961841583252),
 ('zaniness', 0.7353197336196899)]

In [ ]:
glove_model.most_similar('memoryy')

KeyError: "Key 'memoryy' not present in vocabulary"